[倫敦]
========
將上一筆電量加入至模型, 使用Sliding Window，
========
並加入 Self Attention/Attention 架構於LSTM 模型中
========
以分類的資料形式加入模型
========

In [34]:
# Import necessary libraries [B]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import statsmodels.api as sm  # seasonal trend decomposition
import datetime
from statsmodels.graphics import tsaplots   # autocorrelation
# import pymysql
from tensorflow.keras.optimizers import Adam #
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, TimeDistributed, Flatten, Bidirectional,Dropout, MultiHeadAttention
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from time import time
from sklearn import preprocessing, metrics
from joblib import dump, load
from datetime import datetime, timedelta
from scipy.interpolate import lagrange
from scipy.sparse import csr_matrix
from keras_self_attention import SeqSelfAttention
# from attention import Attention
from tensorflow.keras.utils import plot_model
%matplotlib inline
# Import necessary libraries [E]

In [37]:
# 參數定義 [B]
today_object = datetime.now()
today = today_object.strftime("%Y-%m-%d-%H-%M-%S")

wind_dir_zscore_to_path = './../London-parameter-storage4/zscore/wind_dir-zscore-{}.bin'.format(today)
temperature_zscore_to_path = './../London-parameter-storage4/zscore/temperature-zscore-{}.bin'.format(today)
T_Max_zscore_to_path = './../London-parameter-storage4/zscore/T_Max-zscore-{}.bin'.format(today)
dewpoint_zscore_to_path = './../London-parameter-storage4/zscore/dewpoint-zscore-{}.bin'.format(today)
pres_zscore_to_path = './../London-parameter-storage4/zscore/pres-zscore-{}.bin'.format(today)
apparent_temp_zscore_to_path = './../London-parameter-storage4/zscore/apparent_temp-zscore-{}.bin'.format(today)
wind_speed_zscore_to_path = './../London-parameter-storage4/zscore/wind_speed-zscore-{}.bin'.format(today)
day_of_year_zscore_to_path = './../London-parameter-storage4/zscore/day_of_year-zscore-{}.bin'.format(today)
data_week_zscore_to_path = './../London-parameter-storage4/zscore/data_week-zscore-{}.bin'.format(today)
data_hour_zscore_to_path = './../London-parameter-storage4/zscore/data_hour-zscore-{}.bin'.format(today)
data_minute_zscore_to_path = './../London-parameter-storage4/zscore/data_minute-zscore-{}.bin'.format(today)
minutes_of_the_day_zscore_to_path = './../London-parameter-storage4/zscore/minutes_of_the_day-zscore-{}.bin'.format(today)
energy_zscore_to_path = './../London-parameter-storage4/zscore/energy-zscore-{}.bin'.format(today)

data_weekday_encode_to_path = './../London-parameter-storage4/zscore/data_weekday-encode-{}.bin'.format(today)
period_type_encode_to_path = './../London-parameter-storage4/zscore/period_type-encode-{}.bin'.format(today)

simple_LSTM_modal_save_path = './../London-parameter-storage4/modal/simple-LSTM-modal-{}.h5'.format(today)
stacked_LSTM_modal_save_path = './../London-parameter-storage4/modal/stacked-LSTM-modal-{}.h5'.format(today)
bidirectional_LSTM_modal_save_path = './../London-parameter-storage4/modal/bidirectional-LSTM-modal-{}.h5'.format(today)
simple_LSTM_kmeans_modal_save_path = './../London-parameter-storage4/modal/simple-LSTM-kmeans-modal-{}.h5'.format(today)
stacked_LSTM_kmeans_modal_save_path = './../London-parameter-storage4/modal/stacked-LSTM-kmeans-modal-{}.h5'.format(today)
bidirectional_LSTM_kmeans_modal_save_path = './../London-parameter-storage4/modal/bidirectional-LSTM-kmeans-modal-{}.h5'.format(today)


LSTM_modal_selfattention_4t_save_path = './../London-parameter-storage4/modal/simple-LSTM-modal-self-attention-4t-{}.h5'.format(today)
LSTM_modal_selfattention_48t_save_path = './../London-parameter-storage4/modal/simple-LSTM-modal-self-attention-48t-{}.h5'.format(today)
LSTM_modal_selfattention_96t_save_path = './../London-parameter-storage4/modal/simple-LSTM-modal-self-attention-96t-{}.h5'.format(today)
LSTM_modal_selfattention_192t_save_path = './../London-parameter-storage4/modal/simple-LSTM-modal-self-attention-192t-{}.h5'.format(today)



zscore_pid_path = './../London-parameter-storage4/pid/zscore-pid.pid'
encode_pid_path = './../London-parameter-storage4/pid/encode-pid.pid'
modal_pid_path = './../London-parameter-storage4/pid/modal-pid.pid'
stacked_modal_pid_path = './../London-parameter-storage4/pid/stacked-modal-pid.pid'
bidirectional_modal_pid_path = './../London-parameter-storage4/pid/bidirectional-modal-pid.pid'
simple_kmeans_modal_pid_path = './../London-parameter-storage4/pid/simple-kmeans-modal-pid.pid'
stacked_kmeans_modal_pid_path = './../London-parameter-storage4/pid/stacked-kmeans-modal-pid.pid'
bidirectional_kmeans_modal_pid_path = './../London-parameter-storage4/pid/bidirectional-kmeans-modal-pid.pid'

kmeans_pid_path = './../London-parameter-storage4/pid/kmeans-pid.pid'

In [7]:
begin_date = '2011-11-23'
end_date = '2014-02-28'
train_begin_date = '2011-11-23'
train_end_date = '2013-07-02'
predict_begin_date = '2013-07-03'
predict_end_date = '2014-02-08'
# 參數定義 [E]

In [8]:
# 正規化最大最小值 [B]
temperature_range = np.array([-10, 40])
dewpoint_range = np.array([-10, 30])
pres_range = np.array([0, 1500])
apparent_range = np.array([-10, 40])
wind_speed_range = np.array([0, 20])
wind_dir_range = np.array([0, 360])
day_of_year_range = np.array([1, 366])
week_range = np.array([1, 53])
hour_range = np.array([0, 24])
minute_range = np.array([0, 60])
minutes_of_the_day_range = np.array([0, 86400])
energy_range = np.array([0, 10])
# 正規化最大最小值 [B]

In [9]:
# 參數轉成監督式參數 [B]
# The function is defined with default parameters so that if you call it with just your data, it will construct a DataFrame with t-1 
# as X and t as y

def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df = pd.DataFrame(data)
    df_columns_name = df.columns.values
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (df_columns_name[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg
# 參數轉成監督式參數 [E]

In [14]:
# 將測試集與訓練集分割成想要的 sliding window 大小 [B]
def create_dataset(train_features, train_labels, look_back=1):
    X_train, y_train = [], []
    for i in range(train_labels.shape[0] - (look_back-1)):
        X_train.append(train_features.iloc[i:i+look_back].values)
        y_train.append(train_labels.iloc[i + (look_back-1)])
    X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1,1)
    return X_train, y_train
# 將測試集與訓練集分割成想要的 sliding window 大小 [E]

In [10]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = '　' ,  user = '　' ,  passwd = "　" ,  db = 'london_smart_meters' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `hd`.`id`, `hd`.`energy`, `whd`.`windBearing`, `whd`.`temperature`, `whd`.`T_Max`, `whd`.`dewPoint`, `whd`.`pressure`, 
                    `whd`.`apparentTemperature`, `whd`.`windSpeed`, `whd`.`humidity`, `hd`.`day_of_year`, `hd`.`data_week`, 
                    `hd`.`data_date`, `hd`.`data_hour`, `hd`.`data_minute`, `hd`.`data_weekday`,`hd`.`minutes_of_the_day`, `hd`.`datetime`
                    FROM `halfhourly_avg` AS `hd`
                    INNER JOIN `weather_hourly_darksky` AS `whd` ON(`hd`.`data_date` = `whd`.`data_date` AND `hd`.`data_hour` = `whd`.`data_hour`)
                    WHERE `hd`.`datetime` >= '{}' AND `hd`.`datetime` < '{}'
                    ORDER BY `hd`.`datetime` '''.format(begin_date, end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    timestamp = pd.to_datetime(demand_dataframe.datetime, infer_datetime_format=True).values.astype(float)
    demand_dataframe['timestamp'] = timestamp.tolist()
    demand_dataframe = demand_dataframe.set_index('datetime')
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [12]:
# 刪除不需要的欄位 [B]
demand_dataframe = demand_dataframe.drop(columns=['id', 'data_date'])
# 刪除不需要的欄位 [E]

In [13]:
demand_dataframe[demand_dataframe.isnull().any(axis=1)]

,energy,windBearing,temperature,T_Max,dewPoint,pressure,apparentTemperature,windSpeed,humidity,day_of_year,data_week,data_hour,data_minute,data_weekday,minutes_of_the_day,timestamp
datetime,,,,,,,,,,,,,,,,
2012-05-07 18:00:00,0.295,189,12.99,14.48,8.29,None,12.99,4.22,0.73,128,19,18,0,1,1080,1.336414e+18
2012-05-07 18:30:00,0.296,189,12.99,14.48,8.29,None,12.99,4.22,0.73,128,19,18,30,1,1110,1.336415e+18
2012-10-26 15:00:00,0.234,21,7.29,15.54,3.68,None,4.21,4.99,0.78,300,43,15,0,5,900,1.351264e+18
2012-10-26 15:30:00,0.249,21,7.29,15.54,3.68,None,4.21,4.99,0.78,300,43,15,30,5,930,1.351265e+18
2013-01-03 07:00:00,0.169,250,9.65,11.41,8.67,None,7.07,5.21,0.94,3,1,7,0,4,420,1.357196e+18
2013-01-03 07:30:00,0.193,250,9.65,11.41,8.67,None,7.07,5.21,0.94,3,1,7,30,4,450,1.357198e+18
2013-06-09 13:00:00,0.225,48,13.42,14.17,7.76,None,13.42,5.07,0.69,160,23,13,0,7,780,1.370783e+18
2013-06-09 13:30:00,0.220,48,13.42,14.17,7.76,None,13.42,5.07,0.69,160,23,13,30,7,810,1.370785e+18
2013-08-01 00:00:00,0.140,193,19.16,32.40,16.78,None,19.38,2.80,0.86,213,31,0,0,4,0,1.375315e+18


In [15]:
# 補缺失值 [B]
demand_dataframe = demand_dataframe.interpolate(method="linear") 
# 補缺失值 [E]

In [16]:
# 刪除缺失值 [B]
demand_dataframe = demand_dataframe.dropna()
# 刪除缺失值 [E]

In [38]:
# 套函式轉為監督式資料 [B]
reframed = series_to_supervised(demand_dataframe, 1, 1)
# 套函式轉為監督式資料 [E]

In [39]:
# 資料正規化 [B]
# 宣告實例 [B]
wind_dir_zscore = preprocessing.MinMaxScaler()
temperature_zscore = preprocessing.MinMaxScaler()
T_Max_zscore = preprocessing.MinMaxScaler() 
dewpoint_zscore = preprocessing.MinMaxScaler()
pres_zscore = preprocessing.MinMaxScaler()
apparent_temp_zscore = preprocessing.MinMaxScaler()
wind_speed_zscore = preprocessing.MinMaxScaler()
day_of_year_zscore = preprocessing.MinMaxScaler() 
data_week_zscore = preprocessing.MinMaxScaler() 
data_hour_zscore = preprocessing.MinMaxScaler() 
data_minute_zscore = preprocessing.MinMaxScaler() 
minutes_of_the_day_zscore = preprocessing.MinMaxScaler() 
energy_zscore = preprocessing.MinMaxScaler()
# 宣告實例 [E]
# 資料 reshape [B]
wind_dir_reshape_data = reframed['windBearing(t)'].values.reshape(-1, 1)
temperature_reshape_data = reframed['temperature(t)'].values.reshape(-1, 1)
T_Max_reshape_data = reframed['T_Max(t)'].values.reshape(-1, 1)
dewpoint_reshape_data = reframed['dewPoint(t)'].values.reshape(-1, 1)
pres_reshape_data = reframed['pressure(t)'].values.reshape(-1, 1)
apparent_temp_reshape_data = reframed['apparentTemperature(t)'].values.reshape(-1, 1)
wind_speed_reshape_data = reframed['windSpeed(t)'].values.reshape(-1, 1)
day_of_year_reshape_data = reframed['day_of_year(t)'].values.reshape(-1, 1)
data_week_reshape_data = reframed['data_week(t)'].values.reshape(-1, 1)
data_hour_reshape_data = reframed['data_hour(t)'].values.reshape(-1, 1)
data_minute_reshape_data = reframed['data_minute(t)'].values.reshape(-1, 1)
minutes_of_the_day_reshape_data = reframed['minutes_of_the_day(t)'].values.reshape(-1, 1)
last_energy_reshape_data = reframed['energy(t-1)'].values.reshape(-1, 1)
energy_reshape_data = reframed['energy(t)'].values.reshape(-1, 1)
# 資料 reshape [E]
# 部分計算 [B]
wind_dir_zscore.partial_fit(wind_dir_range.reshape(-1, 1))
temperature_zscore.partial_fit(temperature_range.reshape(-1, 1))
T_Max_zscore.partial_fit(temperature_range.reshape(-1, 1))
dewpoint_zscore.partial_fit(dewpoint_range.reshape(-1, 1))
pres_zscore.partial_fit(pres_range.reshape(-1, 1))
apparent_temp_zscore.partial_fit(temperature_range.reshape(-1, 1))
wind_speed_zscore.partial_fit(wind_speed_range.reshape(-1, 1))
day_of_year_zscore.partial_fit(day_of_year_range.reshape(-1, 1))
data_week_zscore.partial_fit(week_range.reshape(-1, 1))
data_hour_zscore.partial_fit(hour_range.reshape(-1, 1))
data_minute_zscore.partial_fit(minute_range.reshape(-1, 1))
minutes_of_the_day_zscore.partial_fit(minutes_of_the_day_range.reshape(-1, 1))
energy_zscore.partial_fit(energy_range.reshape(-1, 1))
# 部分計算 [E]
# 將部分計算得結果先做正規化 [B]
reframed['windBearing(t)'] = wind_dir_zscore.transform(wind_dir_reshape_data)
reframed['temperature(t)'] = temperature_zscore.transform(temperature_reshape_data)
reframed['T_Max(t)'] = T_Max_zscore.transform(T_Max_reshape_data)
reframed['dewPoint(t)'] = dewpoint_zscore.transform(dewpoint_reshape_data)
reframed['pressure(t)'] = pres_zscore.transform(pres_reshape_data)
reframed['apparentTemperature(t)'] = apparent_temp_zscore.transform(apparent_temp_reshape_data)
reframed['windSpeed(t)'] = wind_speed_zscore.transform(wind_speed_reshape_data)
reframed['day_of_year(t)'] = day_of_year_zscore.transform(day_of_year_reshape_data)
reframed['data_week(t)'] = data_week_zscore.transform(data_week_reshape_data)
reframed['data_hour(t)'] = data_hour_zscore.transform(data_hour_reshape_data)
reframed['data_minute(t)'] = data_minute_zscore.transform(data_minute_reshape_data)
reframed['minutes_of_the_day(t)'] = minutes_of_the_day_zscore.transform(minutes_of_the_day_reshape_data)
reframed['energy(t-1)'] = energy_zscore.transform(last_energy_reshape_data)
reframed['energy_normalization'] = energy_zscore.transform(energy_reshape_data)
# 將部分計算得結果先做正規化 [E]
# 資料正規化 [E]

In [40]:
# 儲存正規化模型 [B]
dump(wind_dir_zscore,wind_dir_zscore_to_path, compress=True)
dump(temperature_zscore, temperature_zscore_to_path, compress=True)
dump(T_Max_zscore, T_Max_zscore_to_path, compress=True) #儲存參數
dump(dewpoint_zscore, dewpoint_zscore_to_path, compress=True)
dump(pres_zscore, pres_zscore_to_path, compress=True)
dump(apparent_temp_zscore, apparent_temp_zscore_to_path, compress=True)
dump(wind_speed_zscore, wind_speed_zscore_to_path, compress=True)
dump(day_of_year_zscore, day_of_year_zscore_to_path, compress=True) #儲存參數
dump(data_week_zscore, data_week_zscore_to_path, compress=True) #儲存參數
dump(data_hour_zscore, data_hour_zscore_to_path, compress=True) #儲存參數
dump(data_minute_zscore, data_minute_zscore_to_path, compress=True) #儲存參數
dump(minutes_of_the_day_zscore, minutes_of_the_day_zscore_to_path, compress=True) #儲存參數
dump(energy_zscore, energy_zscore_to_path, compress=True) #儲存參數
# 儲存正規化模型 [E]

In [41]:
# 儲存檔案最後更新檔案日期 [B]
zscore_pid_write = open(zscore_pid_path, 'w', encoding='utf-8')
zscore_pid_write.write(today)
print(today)
# 儲存檔案最後更新檔案日期 [E]

In [42]:
# 將分類資料轉為 one hot encoding [B]
data_weekday_encoder = preprocessing.OneHotEncoder(sparse=False)
# period_type_encoder = preprocessing.OneHotEncoder(sparse=False)

data_weekday_values = reframed['data_weekday(t)'].values.reshape(len(reframed['data_weekday(t)']), 1)
# period_type_values = dataframe['period_type'].values.reshape(len(dataframe['period_type']), 1)

data_weekday_values = data_weekday_encoder.fit_transform(data_weekday_values)
# period_type_values = period_type_encoder.fit_transform(period_type_values)

data_weekday_encode_dataframe= pd.DataFrame(data_weekday_values)
# period_type_encode_dataframe= pd.DataFrame(period_type_values)

for i in data_weekday_encode_dataframe:
    reframed['data_weekday_{}(t)'.format(i)] = data_weekday_encode_dataframe[i].values
# for i in period_type_encode_dataframe:
#     dataframe['period_type_{}'.format(i)] = period_type_encode_dataframe[i].values
# 將分類資料轉為 one hot encoding [E]

In [43]:
# 儲存 one hot encodeing模型 [B]
dump(data_weekday_encoder, data_weekday_encode_to_path, compress=True)
# dump(period_type_encoder, period_type_encode_to_path, compress=True)

# 儲存 one hot encodeing模型 [E]

In [44]:
# 儲存檔案最後更新檔案日期 [B]
encode_pid_write = open(encode_pid_path, 'w', encoding='utf-8')
encode_pid_write.write(today)
print(today)
# 儲存檔案最後更新檔案日期 [E]

In [45]:
# 將分類資料轉為 one hot encoding [B]
data_weekday_encoder = preprocessing.OneHotEncoder(sparse=False)
# period_type_encoder = preprocessing.OneHotEncoder(sparse=False)

data_weekday_values = reframed['data_weekday(t)'].values.reshape(len(reframed['data_weekday(t)']), 1)
# period_type_values = dataframe['period_type'].values.reshape(len(dataframe['period_type']), 1)

data_weekday_values = data_weekday_encoder.fit_transform(data_weekday_values)
# period_type_values = period_type_encoder.fit_transform(period_type_values)

data_weekday_encode_dataframe= pd.DataFrame(data_weekday_values)
# period_type_encode_dataframe= pd.DataFrame(period_type_values)

for i in data_weekday_encode_dataframe:
    reframed['data_weekday_{}(t)'.format(i)] = data_weekday_encode_dataframe[i].values
# for i in period_type_encode_dataframe:
#     dataframe['period_type_{}'.format(i)] = period_type_encode_dataframe[i].values
# 將分類資料轉為 one hot encoding [E]

In [46]:
# 設定要選取的欄位，此參數用意在確保所有所需要的欄位與欄位的順序一致 [B]
selected_column = ['windBearing(t)', 'temperature(t)', 'T_Max(t)', 'dewPoint(t)', 'pressure(t)', 'apparentTemperature(t)', 'windSpeed(t)', 'humidity(t)', 'day_of_year(t)', 'data_week(t)', 'data_hour(t)',
                   'data_minute(t)', 'minutes_of_the_day(t)', 'energy(t-1)', 'data_weekday_0(t)',
                   'data_weekday_1(t)', 'data_weekday_2(t)', 'data_weekday_3(t)', 'data_weekday_4(t)',
                   'data_weekday_5(t)', 'data_weekday_6(t)']
selected_column_noweather = ['day_of_year(t)', 'data_week(t)', 'data_hour(t)',
                   'data_minute(t)', 'minutes_of_the_day(t)', 'energy(t-1)', 'data_weekday_0(t)',
                   'data_weekday_1(t)', 'data_weekday_2(t)', 'data_weekday_3(t)', 'data_weekday_4(t)',
                   'data_weekday_5(t)', 'data_weekday_6(t)']
# 設定要選取的欄位，此參數用意在確保所有所需要的欄位與欄位的順序一致 [E]
# 將資料拆分成訓練集語測式集 [B]
all_train_features = reframed.loc[train_begin_date:train_end_date, selected_column]
all_train_labels = reframed.loc[train_begin_date:train_end_date, 'energy_normalization']

all_predict_features = reframed.loc[predict_begin_date:predict_end_date, selected_column]

all_train_features_no_weather = reframed.loc[train_begin_date:train_end_date, selected_column_noweather]
all_train_labels_no_weather = reframed.loc[train_begin_date:train_end_date, 'energy_normalization']

all_predict_features_no_weather = reframed.loc[predict_begin_date:predict_end_date, selected_column_noweather]

all_predict_labels = reframed.loc[predict_begin_date:predict_end_date, 'energy_normalization']
# 將資料拆分成訓練集語測式集 [E]

In [51]:
# 將所有數值轉成 float32 格式 [B]
all_train_features = all_train_features.astype('float32')
all_train_labels = all_train_labels.astype('float32')

all_predict_features = all_predict_features.astype('float32')
all_predict_labels = all_predict_labels.astype('float32')
# 將所有數值轉成 float32 格式 [E]

In [54]:
# 將資料分割成 Sliding Window [B]
look_back = 96
trainX, trainY = create_dataset(all_train_features, all_train_labels, look_back)
predictX, predictY = create_dataset(all_predict_features, all_predict_labels, look_back)

# 將資料分割成 Sliding Window [E]

In [ ]:
''' 使用 Vanilla LSTM (many to one) + Self Attention 來訓練模型 [B]'''
# 定義模型架構 [B]
vanilla_model = Sequential()
vanilla_model.add(LSTM(50, input_shape=(look_back,trainX.shape[2]), return_sequences=True)) #X_train.shape[1],X_train.shape[2]
vanilla_model.add(SeqSelfAttention(attention_activation='softmax'))
# vanilla_model.add(Attention(50))
vanilla_model.add(Flatten())
vanilla_model.add(Dense(1))
vanilla_model.compile(loss='mse', optimizer='Adam') 
vanilla_model.summary()
# plot_model(vanilla_model, to_file='Vanilla_LSTM_graph_20220729.png')
# 定義模型架構 [E]
# 開始訓練 [B]
history = vanilla_model.fit(trainX, trainY, epochs=300, batch_size=60, shuffle=False) #(50,72)
# 開始訓練 [E]
# 評估模型 [B]
scores = vanilla_model.evaluate(trainX, trainY)
# 評估模型 [E]
''' 使用 Vanilla LSTM (many to one) + Self Attention 來訓練模型 [E]'''

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 192, 50)           14400     
                                                                 
 seq_self_attention_4 (SeqSe  (None, 192, 50)          3265      
 lfAttention)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 9600)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 9601      
                                                                 
Total params: 27,266
Trainable params: 27,266
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
467/467 [==============================] - 373s 795ms/step - loss: 0.0173
Epoch 2/300
467/467 [=========

467/467 [==============================] - 371s 795ms/step - loss: 4.5362e-05
Epoch 81/300
467/467 [==============================] - 370s 792ms/step - loss: 4.5361e-05
Epoch 82/300
467/467 [==============================] - 373s 799ms/step - loss: 4.5360e-05
Epoch 83/300
467/467 [==============================] - 371s 794ms/step - loss: 4.5358e-05
Epoch 84/300
467/467 [==============================] - 372s 798ms/step - loss: 4.5354e-05
Epoch 85/300
467/467 [==============================] - 371s 795ms/step - loss: 4.5351e-05
Epoch 86/300
467/467 [==============================] - 372s 796ms/step - loss: 4.5347e-05
Epoch 87/300
467/467 [==============================] - 373s 799ms/step - loss: 4.5345e-05
Epoch 88/300
467/467 [==============================] - 372s 797ms/step - loss: 4.5343e-05
Epoch 89/300
467/467 [==============================] - 372s 796ms/step - loss: 4.5342e-05
Epoch 90/300
467/467 [==============================] - 370s 793ms/step - loss: 4.5341e-05
Epoch 91/300

In [ ]:
# 用訓練好的模型來預測 [B]
testPredict1 = vanilla_model.predict(predictX)
# 用訓練好的模型來預測 [E]

In [ ]:
# 將預測值與實際值 inverse 正規化 [B]
testPredict1 = energy_zscore.inverse_transform(testPredict1.reshape(1, -1))
y_test_true = energy_zscore.inverse_transform(predictY.reshape(1, -1))
# 將預測值與實際值 inverse 正規化 [E]

In [ ]:
# 將預測值與實際值做圖表 [B]
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([0,150])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true[0], c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict1[0], c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()
# 將預測值與實際值做圖表 [E]

In [ ]:
# 評估模型好壞 [B]
metrics.mean_absolute_error(y_test_true[0],testPredict1[0])
# 評估模型好壞 [E]

In [ ]:
# 儲存模型 [B]
vanilla_model.save(LSTM_modal_selfattention_192t_save_path)  # creates a HDF5 file 'my_model.h5'
# 儲存模型 [E]

------------------------------------

-----------------------------

In [120]:
# Vanilla LSTM (many to one)
vanilla_model_no_weather = Sequential()
vanilla_model_no_weather.add(LSTM(50, input_shape=(train_features2_no_weather.shape[1],train_features2_no_weather.shape[2]))) #X_train.shape[1],X_train.shape[2]
vanilla_model_no_weather.add(Dense(1, activation='relu'))
opt = Adam(learning_rate=0.0003)
vanilla_model_no_weather.compile(loss='mse', optimizer=opt) 
vanilla_model_no_weather.summary()
# plot_model(model, to_file='Vanilla_LSTM_graph.png')
history = vanilla_model_no_weather.fit(train_features2_no_weather, all_train_labels_no_weather, epochs=300, batch_size=48, shuffle=False) #(50,72)
scores = vanilla_model_no_weather.evaluate(train_features2_no_weather, all_train_labels_no_weather)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 50)                12600     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
588/588 [==============================] - 2s 1ms/step - loss: 2.8985e-04
Epoch 2/300
588/588 [==============================] - 1s 1ms/step - loss: 2.5472e-04
Epoch 3/300
588/588 [==============================] - 1s 1ms/step - loss: 2.5397e-04
Epoch 4/300
588/588 [==============================] - 1s 2ms/step - loss: 2.5399e-04
Epoch 5/300
588/588 [==============================] - 1s 1ms/step - loss: 2.5400e-04
Epoch 6/300
588/

588/588 [==============================] - 1s 1ms/step - loss: 2.1725e-05
Epoch 181/300
588/588 [==============================] - 1s 1ms/step - loss: 2.1732e-05
Epoch 182/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1739e-05
Epoch 183/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1743e-05
Epoch 184/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1747e-05
Epoch 185/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1749e-05
Epoch 186/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1750e-05
Epoch 187/300
588/588 [==============================] - 1s 1ms/step - loss: 2.1749e-05
Epoch 188/300
588/588 [==============================] - 1s 1ms/step - loss: 2.1749e-05
Epoch 189/300
588/588 [==============================] - 1s 1ms/step - loss: 2.1748e-05
Epoch 190/300
588/588 [==============================] - 1s 2ms/step - loss: 2.1746e-05
Epoch 191/300
588/588 [=======================

In [121]:
predict_features2_no_weather.shape

(10590, 1, 12)

In [122]:
testPredict1 = vanilla_model_no_weather.predict(predict_features2_no_weather)

In [127]:
vanilla_model_no_weather.save('./../London-parameter-storage4/modal/simple-LSTM-modal-no-weather-20220715.h5')  # creates a HDF5 file 'my_model.h5'

------------------------------------